# Setup

In [1]:
%%capture
! pip install datasets metrics transformers wandb

In [2]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import BertModel, BertTokenizer
from transformers.file_utils import ModelOutput
from datasets import Dataset
from torch import nn
import pandas as pd
import torch
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Custom BERT model

In [10]:
class BERTCNN(nn.Module):

    def __init__(self, model_name='bert-base-uncased', device='cuda'):
        super(BERTCNN, self).__init__()
        self.bert = BertModel.from_pretrained(model_name).to(device)

        self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding='valid').to(device)
        self.relu = nn.ReLU()
        # change the kernel size either to (3,1), e.g. 1D max pooling
        # or remove it altogether
        self.pool = nn.MaxPool2d(kernel_size=(3, 1), stride=1).to(device)
        self.dropout = nn.Dropout(0.1)
        # be careful here, this needs to be changed according to your max pooling
        # without pooling: 443, with 3x1 pooling: 416
        # FC
        self.fc = nn.Linear(598, 3).to(device) ## 416, 66004???
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1).to(device)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            output_hidden_states=True)
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in outputs[2]]), 0), 0, 1)

        x = self.dropout(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)

        x = self.dropout(x)
        x = self.flat(x)
        x = self.dropout(x)
        x = self.fc(x)
        c = self.softmax(x)

        # Clean cache
        gc.collect()
        torch.cuda.empty_cache()
        del outputs

        # Compute loss
        loss = None
        if labels is not None:
            ce_loss = nn.CrossEntropyLoss()
            loss = ce_loss(c, labels)

        return ModelOutput({
            'loss': loss,
            'last_hidden_state': c
        })

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
model = BERTCNN(model_name)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

# Pre-procesing

In [6]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels

    inputs = tokenizer(
        batch["1"],
        max_length=50,
        padding="max_length",
        truncation=True,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    labels = list(map(lambda x: int(x.split('__label__')[1]), batch['0']))
    batch["label"] = labels

    return batch

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate f1 using sklearn's function
  f_score = f1_score(labels, preds, pos_label=0, average='binary')
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, pos_label=0, average='binary')
  recall = recall_score(labels, preds, pos_label=0, average='binary')
  return {
      'f1_score': f_score,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall
  }
    
tokenize_batch_size = 2048
df = pd.read_csv('/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/labeledEligibilitySample1000000.csv', sep='\t', header=None)

In [7]:
train_ds = Dataset.from_pandas(df)

train_ds = train_ds.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=tokenize_batch_size,
    remove_columns=["0", "1"]
)

train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

  0%|          | 0/489 [00:00<?, ?ba/s]

In [8]:
train_testvalid = train_ds.train_test_split(train_size=0.9)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# Training and evaluation

In [11]:
training_args = TrainingArguments(
    output_dir='./CNN_CTC_results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=128,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    fp16=True,                       # enable fp16 apex training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    report_to="wandb"
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_testvalid['train'],         # training dataset
    eval_dataset=test_valid['train'],          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

trainer.train()

preds = trainer.predict(test_valid['test'])

preds[2]

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
Using amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 900000
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 14064
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,F1 Score,Accuracy,Precision,Recall
400,0.522600,0.311992,0.874965,0.875120,0.883600,0.866497
800,0.296400,0.262327,0.891582,0.895320,0.933140,0.853568
1200,0.257600,0.235711,0.912008,0.911040,0.909776,0.914251
1600,0.240900,0.226032,0.908008,0.910440,0.941828,0.876532
2000,0.225400,0.209773,0.918621,0.919080,0.931891,0.905723
2400,0.211300,0.215528,0.918779,0.920280,0.944768,0.894182
2800,0.207600,0.199680,0.923408,0.924540,0.945775,0.902074
3200,0.201900,0.193069,0.925392,0.925740,0.937811,0.913299
3600,0.192500,0.191086,0.929916,0.930000,0.939054,0.920953
4000,0.186100,0.180500,0.932717,0.932500,0.937670,0.927815


***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./CNN_CTC_results/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./CNN_CTC_results/checkpoint-800
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./CNN_CTC_results/checkpoint-1200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./CNN_CTC_results/checkpoint-1600
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128
Saving model checkpoint to ./CNN_CTC_results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only savin

{'test_accuracy': 0.95522,
 'test_f1_score': 0.9548306401178155,
 'test_loss': 0.12351412326097488,
 'test_precision': 0.9629313151041666,
 'test_recall': 0.9468651222342256,
 'test_runtime': 114.4118,
 'test_samples_per_second': 437.018,
 'test_steps_per_second': 3.417}

In [12]:
import torch
torch.save(trainer.model, '/content/drive/MyDrive/PoliTo/Deep Natural Language Procesing/Project/model.model')